In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!git clone https://github.com/long-practice/mlflow-basecode.git

Cloning into 'mlflow-basecode'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 92 (delta 36), reused 60 (delta 17), pack-reused 0
Receiving objects: 100% (92/92), 125.98 KiB | 969.00 KiB/s, done.
Resolving deltas: 100% (36/36), done.


In [3]:
%cd mlflow-basecode

/content/mlflow-basecode


In [4]:
import pandas as pd
import numpy as np

train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [5]:
!pip install optuna --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 5.3 MB/s eta 0:00:00


In [6]:
from src.pipeline import Pipeline
from preprocessing.preprocess import Preprocess

In [7]:
pipe = Pipeline(train, test)
pipe.run()

INFO:preprocess:Preprocessing Set Missing Value
INFO:preprocess:Preprocessing Set Missing Value
INFO:preprocess:Preprocessing Remove Outlier
INFO:preprocess:column name is : id
INFO:preprocess:lower_bound is : -2703.0
INFO:preprocess:upper_bound is : 8109.0
INFO:preprocess:----------------------------------------
INFO:preprocess:column name is : CementComponent
INFO:preprocess:lower_bound is : -28.25000000000003
INFO:preprocess:upper_bound is : 616.95
INFO:preprocess:----------------------------------------
INFO:preprocess:column name is : BlastFurnaceSlag
INFO:preprocess:lower_bound is : -183.89999999999998
INFO:preprocess:upper_bound is : 306.5
INFO:preprocess:----------------------------------------
INFO:preprocess:column name is : FlyAshComponent
INFO:preprocess:lower_bound is : -118.5
INFO:preprocess:upper_bound is : 197.5
INFO:preprocess:----------------------------------------
INFO:preprocess:column name is : WaterComponent
INFO:preprocess:lower_bound is : 149.75
INFO:preprocess

(4325, 12) (4325,) (1082, 12) (1082,)


[I 2023-10-03 14:37:54,123] Trial 0 finished with value: 12.570164445000783 and parameters: {'max_depth': 19, 'learning_rate': 0.012836978417585555, 'n_estimators': 838, 'min_child_weight': 3, 'gamma': 0.9281099098609681, 'subsample': 0.8114687514792496, 'colsample_bytree': 0.031272227746281585, 'reg_alpha': 0.6134460881079817, 'reg_lambda': 0.45232933613267573}. Best is trial 0 with value: 12.570164445000783.
[I 2023-10-03 14:37:56,923] Trial 1 finished with value: 16.07266977807542 and parameters: {'max_depth': 4, 'learning_rate': 0.6925402664659014, 'n_estimators': 721, 'min_child_weight': 6, 'gamma': 0.13936829612065565, 'subsample': 0.20838027494182218, 'colsample_bytree': 0.1627284101028015, 'reg_alpha': 0.3036625009095553, 'reg_lambda': 0.7937827615372616}. Best is trial 0 with value: 12.570164445000783.
[I 2023-10-03 14:38:02,697] Trial 2 finished with value: 18.160063193070485 and parameters: {'max_depth': 5, 'learning_rate': 0.5332305779752603, 'n_estimators': 656, 'min_child

In [8]:
import pickle

xgb_model = None
with open('./artifact/XGB_model.pkl', 'rb') as f:
    xgb_model = pickle.load(f)
submission = pd.read_csv('./data/sample_submission.csv')
submission['Strength'] = xgb_model.predict(test)

In [9]:
submission

,id,Strength
0,5407,45.266056
1,5408,17.872177
2,5409,27.020905
3,5410,51.821815
4,5411,41.561089
...,...,...
3600,9007,64.433411
3601,9008,51.081619
3602,9009,45.818233
3603,9010,40.691109


In [10]:
import os

n = 0
submission_file_name_list = sorted([fname for fname in os.listdir('./prediction/') if fname.split('.')[-1] == 'csv'])
if submission_file_name_list:
    remove_tail = submission_file_name_list[-1].rstrip('.csv')
    n = int(remove_tail.split('_')[-1])
    n += 1

submission_path = './prediction/'
save_file_name = f'submission_{n}.csv'
file_path = os.path.join(submission_path, save_file_name)
submission.to_csv(file_path, index=False)